| Lang | Num Docs | Num Sys | Num Sys * Num Docs | Total |
| -------- | ------- | -------- | ------- | -------- |
| en_nl | 27 | 5 | 135 | 3830 |
| en_pt | 41 | 5 | 205 | 4700 |
| en_de | 36 | 7 | 252 | 6629 |
| en_ko | 24 | 4 | 96 |  3648 |
| en_fr | 37 | 6 | 222 | 6324 |

In [13]:
import pandas as pd
import collections
import json

lang_dict = {"en": "eng", "de": "deu", "pt": "por", "fr": "fra", "ko": "kor", "nl": "nld"}

for lp in ["en-de", "en-fr", "en-ko", "en-nl", "en-pt"]:

  tgt_lang = lp.split("-")[1]
  scores_df = pd.read_csv(f"human_eval/docsqm2{lang_dict[tgt_lang]}_ann.csv")

  scores_df.drop_duplicates(subset=['model_app'], keep='last', inplace=True)

  print(f"language Pair: {lp} #(Document-level) {len(scores_df[scores_df.is_doc==True])} #(Sentence-level) {len(scores_df[scores_df.is_doc==False])} #Total: {len(scores_df)}")
  print("============= Document-level ===========")
  print(scores_df[scores_df.is_doc==True].groupby("model")["score"].mean())

  main_df = pd.read_csv(f"all_submissions/{lp}.csv")
  all_df = []
  for gr, gr_df in main_df.groupby("doc_id"):
    gr_df["seg_id"] = list(range(0, len(gr_df)))
    all_df.append(gr_df)
  all_df = pd.concat(all_df)

  assert collections.Counter(scores_df["model"].unique()) == collections.Counter(all_df.columns[8:-1])

  all_df = all_df.melt(id_vars=['source_language', 'target_language', 'source', 'reference', 'doc_id', 'client_id', 'sender', 'tags', 'seg_id'],
                       value_vars=all_df.columns[8:-1], var_name="model", value_name="output")
  all_df.rename(columns={"seg_id": "sent_id"}, inplace=True)

  new_df = scores_df[scores_df.is_doc==False].merge(all_df, on=["doc_id", "sent_id", "model"])
  assert len(scores_df[scores_df.is_doc==False]) == len(new_df)

  for sub_lp, sub_lp_df in new_df.groupby(["source_language", "target_language"]):
    print("============== Sentence-level ===========")
    print(f"Language Direction: {sub_lp[0]} ->  {sub_lp[1]}")
    print(sub_lp_df.groupby("model")["score"].mean())

  print("\n\n")

language Pair: en-de #(Document-level) 252 #(Sentence-level) 6377 #Total: 6629
============= Document-level ===========
model
submission_ADAPT            75.750000
submission_DCUGenNLP        77.027778
submission_HW-TSC           81.194444
submission_SheffieldGATE    75.722222
submission_baseline         74.500000
submission_clteam           78.611111
submission_unbabel+it       84.222222
Name: score, dtype: float64
============== Sentence-level ===========
Language Direction: de ->  en
model
submission_ADAPT            88.827354
submission_DCUGenNLP        88.598655
submission_HW-TSC           90.414798
submission_SheffieldGATE    88.849776
submission_baseline         87.565022
submission_clteam           89.123318
submission_unbabel+it       92.735426
Name: score, dtype: float64
============== Sentence-level ===========
Language Direction: en ->  de
model
submission_ADAPT            82.550538
submission_DCUGenNLP        84.556989
submission_HW-TSC           88.470968
submission_Sheff